![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F08+-+R%2Fdepricated&file=08Tools+-+Prediction+-+Custom.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/08%20-%20R/depricated/08Tools%20-%20Prediction%20-%20Custom.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F08%2520-%2520R%2Fdepricated%2F08Tools%2520-%2520Prediction%2520-%2520Custom.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/08%20-%20R/depricated/08Tools%20-%20Prediction%20-%20Custom.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/08%20-%20R/depricated/08Tools%20-%20Prediction%20-%20Custom.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 08Tools: Prediction - Custom

**IN ACTIVE DEVELOPMENT - NOT COMPLETE**

Serve interactive/online predictions with the model created in the `08b Training Job ...` notebook.  This will use the custom container for `08b` and will show serving locally with docker and on an endpoint with Cloud Run.

**Prerequisites:**
- [08b Training Job - Vertex AI Custom Model - R - Training Pipeline With Custom Container](./08b%20Training%20Job%20-%20Vertex%20AI%20Custom%20Model%20-%20R%20-%20Training%20Pipeline%20With%20Custom%20Container.ipynb)


---
## Setup

### Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (Artifact Registry, Cloud Run, and Cloud Build)

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [267]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

In [268]:
try:
    import google.cloud.run_v2
except ImportError:
    print('You need to pip install google-cloud-run')
    !pip install google-cloud-run -q

### Environment

inputs:

In [269]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [270]:
REGION = 'us-central1'
EXPERIMENT = '08_predictions'
SERIES = '08'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [271]:
from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import service_usage_v1
from google.cloud import run_v2
from google.cloud import storage

import requests
import json
import numpy as np

import multiprocessing

clients:

In [272]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()
gcs = storage.Client()
su_client = service_usage_v1.ServiceUsageClient()
run_client = run_v2.ServicesClient()

parameters:

In [273]:
BUCKET = PROJECT_ID
DIR = f"temp/{EXPERIMENT}"

environment:

In [274]:
!rm -rf {DIR}
!mkdir -p {DIR}

### Enable APIs

Using Cloud Run requires enabling the API for the Google Cloud Project.

Options for enabeling.  In this notebook option 2 is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for Cloud Build and Enable
     - Search for Artifact Registry and Enable
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

#### Cloud Run

In [275]:
cloudrun = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/run.googleapis.com'
    )
).state.name


if cloudrun == 'DISABLED':
    print(f'Cloud Run is currently {cloudrun} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/run.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Run is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Run already enabled for project: {PROJECT_ID}')

Cloud Run already enabled for project: statmike-mlops-349915


---
## Get a Model For Predictions
This project already has a model serving online predictions at a Vertex AI Endpoint.  This section will use the endpoint to retrieve the deployed model and get its information to use for batch prediction methods in this notebook.

### Get Endpoint

[Endpoint Properties and Methods](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Endpoint):

```python
endpoint
endpoint.display_name
endpoint.resource_name
endpoint.traffic_split
endpoint.list_models()
```

In [276]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
endpoint = endpoints[0]

In [277]:
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/1178329019301494784?project=statmike-mlops-349915


### Get Model at Endpoint
Using the model on the endpoint for the current series:

In [278]:
endpoint

resource name: projects/1026793852137/locations/us-central1/endpoints/1178329019301494784

In [279]:
#endpoint.list_models()[0]

In [280]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

### Review Model Information

In [281]:
model.display_name

'08_08b'

In [282]:
model.resource_name

'projects/1026793852137/locations/us-central1/models/model_08_08b'

In [283]:
model.version_id

'1'

In [284]:
model.version_description

'run-20221103124201'

In [285]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_08_08b@1'

In [286]:
model.supported_input_storage_formats

['jsonl', 'bigquery', 'csv', 'tf-record', 'tf-record-gzip', 'file-list']

In [287]:
model.name

'model_08_08b'

In [288]:
model.uri

'gs://statmike-mlops-349915/08/08b/models/20221103124201/model'

In [289]:
model.container_spec

image_uri: "us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/08b"
command: "Rscript"
command: "./code/serve.R"

In [290]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}/versions/{model.version_id}/properties?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_08_08b/versions/1/properties?project=statmike-mlops-349915


#### Review Model Information Using the `aiplatform_v1` Model Client
It may also be helpful to try the [ModelServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.model_service.ModelServiceClient) in version 1 of the client to review the model attributes.  Here is example code for trying this.

Curious about client versions and layers?  Check out this tip document [aiplatform_notes.md](../Tips/aiplatform_notes.md).

In [291]:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
ModelClientv1 = aiplatform_v1.ModelServiceClient(client_options = client_options)

ModelClientv1.get_model(
    name = model.versioned_resource_name
)

name: "projects/1026793852137/locations/us-central1/models/model_08_08b@1"
display_name: "08_08b"
predict_schemata {
}
metadata {
}
training_pipeline: "projects/1026793852137/locations/us-central1/trainingPipelines/6968697824896614400"
container_spec {
  image_uri: "us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/08b"
  command: "Rscript"
  command: "./code/serve.R"
}
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1667479599
  nanos: 253558000
}
update_time {
  seconds: 1667479925
  nanos: 631106000
}
deployed_models {
  endpoint: "projects/1026793852137/locations/us-central1/endpoin

---
## Retrieve Records For Prediction

In [292]:
n = 1000
pred = bq.query(query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE splits='TEST' LIMIT {n}").to_dataframe()

In [293]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST


Remove columns not included as features in the model:

In [294]:
newobs = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')
#newobs[0]

In [295]:
len(newobs)

1000

In [296]:
newobs[0]

{'Time': 35337,
 'V1': 1.0928441854981998,
 'V2': -0.0132303486713432,
 'V3': 1.35982868199426,
 'V4': 2.7315370965921004,
 'V5': -0.707357349219652,
 'V6': 0.8738370029866129,
 'V7': -0.7961301510622031,
 'V8': 0.437706509544851,
 'V9': 0.39676985012996396,
 'V10': 0.587438102569443,
 'V11': -0.14979756231827498,
 'V12': 0.29514781622888103,
 'V13': -1.30382621882143,
 'V14': -0.31782283120234495,
 'V15': -2.03673231037199,
 'V16': 0.376090905274179,
 'V17': -0.30040350116459497,
 'V18': 0.433799615590844,
 'V19': -0.145082264348681,
 'V20': -0.240427548108996,
 'V21': 0.0376030733329398,
 'V22': 0.38002620963091405,
 'V23': -0.16764742731151097,
 'V24': 0.0275573495476881,
 'V25': 0.59211469704354,
 'V26': 0.219695164116351,
 'V27': 0.0369695108704894,
 'V28': 0.010984441006191,
 'Amount': 0.0}

---
## Cloud Run: R Model Server

Cloud Run will be used to host our custom container and handle prediction requests.

### Deploy as Cloud Run Service
This demonstration creates an open service allowing all traffic.  Review documentation for [Cloud Run](https://cloud.google.com/run/docs/overview/what-is-cloud-run) and the [CLOUD SDK CLI sections](https://cloud.google.com/sdk/gcloud/reference/run) for `gcloud run`.


If you have a policy inforced for 'Domain Restricted Sharing' then it may need adjusting for the project to allow this.  This should be done with care and you may wish to only accept authenticated or internal traffic.  Review options for authentication [here](https://cloud.google.com/run/docs/authenticating/overview).

Updated Org Policy:
- Logged in as Admin
- IAM > Organization Policies
    - Changed to Project (not org level)
    - Filter 'Domain Restricted Sharing'
    - Select and Edit
        - Applies to = Customize
        - Policy enforcement = Replace
        - Rules = Allow all
    - Save

View the Cloud Run Console for this project:

In [297]:
print(f'https://console.cloud.google.com/run?project={PROJECT_ID}')

https://console.cloud.google.com/run?project=statmike-mlops-349915


Create the Cloud Run Service:

In [298]:
!gcloud run deploy endpoint-$SERIES \
    --image=$model.container_spec.image_uri \
    --command='Rscript' \
    --command='./code/serve_cloudrun.R' \
    --port=8080 --region=$REGION \
    --platform=managed \
    --allow-unauthenticated \
    --no-user-output-enabled

Deploying new service...                                                       
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      


In [299]:
!gcloud run services list

   SERVICE      REGION       URL                                          LAST DEPLOYED BY                                     LAST DEPLOYED AT
✔  endpoint-08  us-central1  https://endpoint-08-urlxi72dpa-uc.a.run.app  1026793852137-compute@developer.gserviceaccount.com  2022-11-03T13:24:40.183176Z


In [300]:
services = !gcloud run services list --format="json" --filter=SERVICE:endpoint-$SERIES
services = json.loads("".join(services))[0]
services['status']['url']

'https://endpoint-08-urlxi72dpa-uc.a.run.app'

If you had to adjust a `Domain Restricted Sharing` policy after deployment then this command can update the service to allow all traffic:

In [301]:
#!gcloud run services add-iam-policy-binding --region=us-central1 --member='allUsers' --role=roles/run.invoker endpoint-$SERIES

### Get Predictions Using Cloud Run Service

Prepare in instance (observation) for prediction request:

In [302]:
tryob_json = json.dumps({"instances": newobs[0:20]})
#tryob_json

Make prediction request:

In [303]:
json_response = requests.post(f"{services['status']['url']}/predict", data=tryob_json, headers={"content-type": "application/json"})

In [304]:
json_response

<Response [200]>

In [305]:
print(json_response.text)

{"predictions":[0.0018,0.0004,0.0017,0,0.0005,0.0002,0.0002,0.0026,0.0016,0.0004,0.0001,0.0002,0.0006,0.0003,0.0028,0.0004,0.0025,0.0016,0.0009,0.0001]}


In [306]:
predictions = json.loads(json_response.text)['predictions']
predictions

[0.0018,
 0.0004,
 0.0017,
 0,
 0.0005,
 0.0002,
 0.0002,
 0.0026,
 0.0016,
 0.0004,
 0.0001,
 0.0002,
 0.0006,
 0.0003,
 0.0028,
 0.0004,
 0.0025,
 0.0016,
 0.0009,
 0.0001]

In [307]:
np.argmax(predictions[0])

0

### Remove Service
This command will remove the service.

Alternatively, you could adjust the service to not accept traffic.  Cloud Run will scale down to zero - or only charge when CPU is used (startup, shutdown, and receiving requests) unless `--no-cpu-throttling` is used ([documentation](https://cloud.google.com/run/docs/configuring/cpu-allocation#setting)).

In [308]:
!gcloud run services delete --region=us-central1 --quiet endpoint-$SERIES

Deleting [endpoint-08]...done.                                                 
Deleted service [endpoint-08].


---
## Local (to Notebook): R Model Server

The local notebook environment may have docker installed. For example, Vertex AI Workbench user-managed instances do have docker installed.  In this case the custom container in Artfact Registry created above can be pulled (`docker pull`) and run (`docker run`) here to serve prediction requests from the local compute instance.  This can be great for testing, troubleshooting, or doing local experiments. It can also potentially be helpful when you need to try various prediction requests scenarios for interpretation/explainability like is done in these notebooks:
- [05Tools - Interpretability with LIT.ipynb](./05Tools%20-%20Interpretability%20with%20LIT.ipynb)
- [05Tools - Interpretability with WIT.ipynb](./05Tools%20-%20Interpretability%20with%20WIT.ipynb)

### Pull the custom serving image locally

In [309]:
!docker pull $model.container_spec.image_uri

Using default tag: latest
latest: Pulling from statmike-mlops-349915/statmike-mlops-349915/08b

20708c8b: Already exists 
99c96e17: Already exists 
88561c53: Already exists 
b700ef54: Already exists 
2e8fe530: Already exists 
b700ef54: Already exists 
f3784f8e: Already exists 
e425dd4a: Already exists 
d87928c6: Already exists 
1aa6227e: Already exists 
67d1d89a: Already exists 
c4f64bc7: Already exists 
9623776c: Already exists 
e2bd6d97: Already exists 
db888e6e: Already exists 
53e0732a: Already exists 
ccf524f5: Already exists 
152c4edc: Already exists 
15d705b0: Already exists 
b765933f: Already exists 
cba45088: Already exists 
14991333: Already exists 
1c951564: Already exists 
1fdb112e: Already exists 
2a17da6d: Already exists 
f9bf3ba4: Already exists 
d28a2ea2: Already exists 
6edbb35e: Already exists 
6c418877: Already exists 
71005745: Already exists 
98900aa8: Already exists 
16e31a3f: Already exists 
0cbec6df: Already exists 
c7c348b4: Already exists 
b6070393: Already ex

### Run the serving image locally

The container is going to be run with commands in this notebook.  In order to run the serving while not tying up further exectutions in this notebook, a subprocess will be launched using `multiprocessing`. To learn more about multiprocessing and running tasks from Python in parallel visit the tips notebook [Python Multiprocessing](../Tips/Python%20Multiprocessing.ipynb).

First, build the syntax of the `docker run` command.

In [310]:
command = f'''docker run -t --rm -p 8501:8080 {model.container_spec.image_uri} Rscript ./code/serve_cloudrun.R'''
print(command)

docker run -t --rm -p 8501:8080 us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/08b Rscript ./code/serve_cloudrun.R


Run the command in a subprocess at the local folder of this notebook - use multiprocess.Process():

In [311]:
def docker_runner():
    !{command}
    #!docker run -t -p 8501:8501 -v "/$(pwd)/temp/05_predictions/model:/models/05/01" -e MODEL_NAME=05 tensorflow/serving:2.7.4

def main():
    p = multiprocessing.Process(target=docker_runner)
    p.start()
    return p
    
p = main()

_CE_CONDA               
_CE_M                   
_R_CHECK_COMPILATION_FLAGS_KNOWN_
                        -Wformat -Werror=format-security -Wdate-time
ANACONDA_PYTHON_VERSION
                        3.7
CONDA_DEFAULT_ENV       base
CONDA_EXE               /opt/conda/bin/conda
CONDA_PREFIX            /opt/conda
CONDA_PROMPT_MODIFIER   (base)
CONDA_PYTHON_EXE        /opt/conda/bin/python
CONDA_SHLVL             1
CONTAINER_NAME          r-cpu/4-1
DL_ANACONDA_HOME        /opt/conda
EDITOR                  vi
GSETTINGS_SCHEMA_DIR    /opt/conda/share/glib-2.0/schemas
GSETTINGS_SCHEMA_DIR_CONDA_BACKUP
                        
HOME                    /root
HOSTNAME                fa577292c8a8
LANG                    C.UTF-8
LC_ALL                  C.UTF-8
LD_LIBRARY_PATH         /usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/java-11-openjdk-amd64/lib/server:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
LN_S   

### Get Predictions on Exposed Port

Prepare in instance (observation) for prediction request:

In [313]:
tryob_json = json.dumps({"instances": [newobs[0]]})
tryob_json

'{"instances": [{"Time": 35337, "V1": 1.0928441854981998, "V2": -0.0132303486713432, "V3": 1.35982868199426, "V4": 2.7315370965921004, "V5": -0.707357349219652, "V6": 0.8738370029866129, "V7": -0.7961301510622031, "V8": 0.437706509544851, "V9": 0.39676985012996396, "V10": 0.587438102569443, "V11": -0.14979756231827498, "V12": 0.29514781622888103, "V13": -1.30382621882143, "V14": -0.31782283120234495, "V15": -2.03673231037199, "V16": 0.376090905274179, "V17": -0.30040350116459497, "V18": 0.433799615590844, "V19": -0.145082264348681, "V20": -0.240427548108996, "V21": 0.0376030733329398, "V22": 0.38002620963091405, "V23": -0.16764742731151097, "V24": 0.0275573495476881, "V25": 0.59211469704354, "V26": 0.219695164116351, "V27": 0.0369695108704894, "V28": 0.010984441006191, "Amount": 0.0}]}'

Make prediction request:

In [314]:
json_response = requests.post(f'http://localhost:8501/predict', data=tryob_json, headers={"content-type": "application/json"})

In [315]:
json_response

<Response [200]>

In [316]:
print(json_response.text)

{"predictions":[0.0018]}


In [317]:
predictions = json.loads(json_response.text)['predictions']
predictions

[0.0018]

In [318]:
np.argmax(predictions[0])

0

### Shutdown Serving Container
There are two entities running: a subprocess called `p` and a docker container that was run by the subprocess.  It is not enough to just stop `p` but it might be enough to stop the container and then the subprocess will terminate due to completion.  The command below stop the subprocess `p` and then stop and remove the container.

In [319]:
p.terminate()

In [320]:
p.is_alive()

False

In [321]:
docker = !docker ps -a
docker

['CONTAINER ID   IMAGE                                                                        COMMAND                  CREATED          STATUS          PORTS                    NAMES',
 'fa577292c8a8   us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/08b   "/entrypoint.sh Rscr…"   30 seconds ago   Up 29 seconds   0.0.0.0:8501->8080/tcp   happy_bohr',
 '6e32d0fb9e8a   gcr.io/inverting-proxy/agent                                                 "/bin/sh -c \'/opt/bi…"   9 days ago       Up 9 days                                proxy-agent']

In [322]:
for d in docker:
    if model.container_spec.image_uri in d:
        print(d.split()[-1])
        !docker stop {d.split()[-1]}
        !docker rm {d.split()[0]}

happy_bohr
happy_bohr
Error: No such container: fa577292c8a8


In [323]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED      STATUS      PORTS     NAMES
6e32d0fb9e8a   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   9 days ago   Up 9 days             proxy-agent
